In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import math
import numpy as np

N_FINGERS = 1
PITCH = 2.5 # mm
CLEARANCE = 0.11 # mm
THETA = np.deg2rad(40) # Angle between edges

In [131]:

def make_edge_points(n_fingers, pad_pitch, clearance, theta):
    margin = pad_pitch * 0.15
    size = pad_pitch - clearance 
    
    chord_pitch = (size - margin*2) / (n_fingers * 4.0)
    
    # Total displacement from center of the reference line
    d0 = chord_pitch / 2 / math.tan(theta / 2)
    # displacement offset for actual line to allow for clearance between neighboring electrodes
    h = clearance / 2 / math.sin(theta/2)
    dm_inside = (h - c) / math.tan(math.pi/2 - theta/2)
    dm_outside = (h - c) * math.tan(theta/2)
    displacements = np.array([clearance/2] * 2 + [d0, d0, -d0 + h] * n_fingers + [clearance/2] * 2)
    
    chords = [clearance/2]
    chords.append(clearance/2 + margin - dm_inside)
    chord0 = clearance/2 + margin
    for i in range(n_fingers):
        center_offset = i*4*chord_pitch
        chamfer_offset = h * math.tan(theta/2)
        chords.append(chord0 + center_offset + chord_pitch- chamfer_offset)
        chords.append(chord0 + center_offset + chord_pitch + chamfer_offset)
        chords.append(chord0 + center_offset + chord_pitch * 3)
        
    chords.append(pad_pitch - clearance/2 - margin - dm_outside)
    chords.append(pad_pitch - clearance/2)
    chords = np.array(chords)
    
    return chords, displacements

def make_pad(n_fingers, pad_pitch, clearance, theta, edges):
    """Make polygon for a pad
    
    edges is a list of 4 booleans corresponding to the four edges, top, right, bottom, left. If true, 
    the edge will be drawn with fingers, otherwise it will be flat. 
    """
   
    chords, displacements = make_edge_points(n_fingers, pad_pitch, clearance, theta)
    
    def push_points(newp):
        nonlocal points
        points = np.vstack([points, newp])
    # concatenate four edges
    flat_displacements = np.ones_like(chords) * clearance/2
    if edges[0]:
        points = np.array([chords, displacements]).T
    else:
        points = np.array([chords, flat_displacements]).T
    if edges[1]:
        push_points(np.array([pad_pitch - displacements, chords]).T)
    else: 
        push_points(np.array([pad_pitch - flat_displacements, chords]).T)
    if edges[2]:
        push_points(np.array([pad_pitch - chords, pad_pitch - displacements]).T)
    else:
        push_points(np.array([pad_pitch - chords, pad_pitch - flat_displacements]).T)
    if edges[3]:
        push_points(np.array([displacements, pad_pitch - chords]).T)
    else:
        push_points(np.array([flat_displacements, pad_pitch - chords]).T)
    return points



In [132]:
from matplotlib.patches import Polygon
margin = 0.5

ax = None
fig = plt.figure()

offsets = np.array([[0, 0], [0, PITCH], [PITCH, 0], [PITCH, PITCH]])
p = make_pad(N_FINGERS, PITCH, CLEARANCE, THETA, [True, True, True, True])
ax = fig.add_subplot(111, sharex=ax)

for offset in offsets:
    poly = Polygon(p + offset)
    ax.add_patch(poly)
ax.set_xlim([-margin, PITCH*2+margin])
ax.set_ylim([PITCH*2+margin, -margin ])

# for i, theta in enumerate([90, 75, 60, 45]):
#     THETA = np.deg2rad(theta)
#     offsets = np.array([[0, 0], [0, PITCH], [PITCH, 0], [PITCH, PITCH]])
#     #offsets = [(0, 0)]
#     p = normalized_points()
#     #print(p)
#     ax = fig.add_subplot(221 + i, sharex=ax)

#     for offset in offsets:
#         poly = Polygon(p + offset)
#         ax.add_patch(poly)
#     ax.set_title('THETA = %d deg' % theta)

#     ax.set_xlim([-margin, PITCH*2+margin])
#     ax.set_ylim([-margin, PITCH*2+margin])
#     ax.axis('square')
# plt.tight_layout()

/Users/jeff/venvs/cv4/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(5.5, -0.5)

In [141]:
from KicadModTree import *

edge_permutations = [
    "trbl",
    "t",
    "b",
    "r",
    "l",
    "tr",
    "tl",
    "tb",
    "rb",
    "rl",
    "bl",
    "rbl",
    "trb",
    "trl",
    "tbl",
]
for variant in edge_permutations:
    footprint_name = 'electrode_points_%d_%ddeg_%s' % (N_FINGERS, int(np.rad2deg(THETA)), variant)

    kicad_mod = Footprint(footprint_name)
    kicad_mod.setDescription("Pointy electrode with two 45degree fingers")

    kicad_mod.append(Text(type="reference", text="REF**", at=[0, -3], layer='F.SilkS', hide=True))
    kicad_mod.append(Text(type="value", text="DMFElectrode", at=[1.5, 3], layer='F.Fab'))
    
    edges = ["t" in variant, "r" in variant, "b" in variant, "l" in variant]
    polygon = Polygon(
        nodes=(make_pad(N_FINGERS, PITCH, CLEARANCE, THETA, edges) - np.array([PITCH/2, PITCH/2])).tolist(), 
        layer='F.Cu',
        width=0.0001)
    pad = kicad_mod.append(Pad(
        number=1,
        type=Pad.TYPE_SMT,
        shape=Pad.SHAPE_CUSTOM,
        at=[0, 0],
        size=[0.5, 0.5],
        layers=Pad.LAYERS_SMT,
        primitives=[polygon]))

    # output kicad model
    file_handler = KicadFileHandler(kicad_mod)
    file_handler.writeFile(footprint_name + ".kicad_mod")

In [140]:
# Make reservoir shape A

import KicadModTree as kmt
import matplotlib.patches as patches


L1 = 4.0
L2 = 2.0
L3 = 8.0
L4 = PITCH * 1.5
(chords, displacements) = make_edge_points(N_FINGERS, PITCH, CLEARANCE, THETA)

c = CLEARANCE/2

edges1 = [
    [[c, L1 - c], [c, c],],
    [L1 - displacements,  chords],
    [[L1 - c, c], [PITCH-c, PITCH-c]],
    [displacements, PITCH-chords]

]
edges1 = [np.array(e).T for e in edges1]
poly1 = np.vstack(edges1)


edges2 = [L1 + displacements, PITCH - chords]
edges2 = [(x, y) for x, y in zip(edges2[0], edges2[1])]

edges2 += [
    (L1 + c, -c),
    (L1 - L2, -c),
    (L1 - L2, -L4 + c),
    (L2 + L3, -L4 + c),
    (L2 + L3, PITCH + L4 - c),
    (L1 - L2, PITCH + L4 - c),
    (L1 - L2, PITCH + c),
    (L1 + c, PITCH + c)
]

#edges2 = [np.array(e).T for e in edges2]
poly2 = np.array(edges2)

ax = None
fig = plt.figure()

offsets = np.array([[0, 0], [0, PITCH], [PITCH, 0], [PITCH, PITCH]])
#p = make_pad(N_FINGERS, PITCH, CLEARANCE, THETA, [True, True, True, True])
ax = fig.add_subplot(111, sharex=ax)
ax.add_patch(patches.Polygon(poly1))
ax.add_patch(patches.Polygon(poly2,  color='red'))

ax.set_xlim([-1, 12])
ax.set_ylim([-4, 8])

footprint_name = "reservoir_A_%.2fmm" % PITCH
kicad_mod = kmt.Footprint(footprint_name)
kicad_mod.setDescription("Reservoir A for %.2fmm pitch electrodes" % PITCH)

kicad_mod.append(Text(type="reference", text="REF**", at=[0, -3], layer='F.SilkS', hide=True))
kicad_mod.append(Text(type="value", text="DMFReservoir", at=[1.5, 3], layer='F.Fab'))

fp_poly1 = kmt.Polygon(
    nodes=(poly1 - np.array([PITCH/2, PITCH/2])).tolist(), 
    layer='F.Cu',
    width=0.0001)

kicad_mod.append(kmt.Pad(
    number=1,
    type=Pad.TYPE_SMT,
    shape=Pad.SHAPE_CUSTOM,
    at=[0, 0],
    size=[0.5, 0.5],
    layers=Pad.LAYERS_SMT,
    primitives=[fp_poly1]))

pad2_origin = [L2 + L3/2, 0]
fp_poly2 = kmt.Polygon(
    nodes=(poly2 - np.array([PITCH/2, PITCH/2]) - np.array(pad2_origin)).tolist(), 
    layer='F.Cu',
    width=0.0001)

kicad_mod.append(kmt.Pad(
    number=2,
    type=kmt.Pad.TYPE_SMT,
    shape=kmt.Pad.SHAPE_CUSTOM,
    at=pad2_origin,
    size=[0.5, 0.5],
    layers=kmt.Pad.LAYERS_SMT,
    primitives=[fp_poly2]))

# output kicad model
file_handler = kmt.KicadFileHandler(kicad_mod)
file_handler.writeFile(footprint_name + ".kicad_mod")

/Users/jeff/venvs/cv4/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
# Make reservoir shape B

import matplotlib.patches as patches

(chords, displacements) = make_edge_points(N_FINGERS, PITCH, CLEARANCE, THETA)
c = CLEARANCE/2
# Length of long feeder electrode in units of PITCH 
L1 = 1.5

def make_points_list(xp, yp):
    return [(x, y) for x, y in zip(reversed(xp), reversed(yp))]

jagged_edge = [(x, y) for x, y in zip(chords, displacements)]
####
# Make the output electrode
####
edges1 = make_points_list(displacements, PITCH - chords) + \
    [
        (c, PITCH-c),
        (PITCH-c, PITCH-c),
    ] + \
    make_points_list(PITCH - displacements, chords) + \
    [
        (PITCH - c, c),
        (c, c)
    ]
poly1 = np.array(edges1)

####
# Make the long feeder electrode
####
edges2 = make_points_list(displacements + PITCH, PITCH - chords) + \
    [
        (PITCH + c,  PITCH - c),
    ] + \
    make_points_list((1+L1) * PITCH - displacements, chords) + \
    [
        ((1 + L1) * PITCH - c, c),
        (PITCH + c, c)
    ]
poly2 = np.array(edges2)

####
# Make the "C"
####
# Amount of C overlap with feeder electrode 
L2 = 1.0 * PITCH
# Depth of reservoir
L3 = 3.0 * PITCH
# Thickness of reservoir arms
L4 = 1.5 * PITCH
# Define an origin at the top right corner of the feeder for simplification
x0 = (1 + L1) * PITCH 

x1 = x0 - L2 + c
x2 = x0 + L3 - c
y1 = PITCH + c
y2 = PITCH + L4 - c
y3 = -(L4 - c)
y4 = -c
edges3 = make_points_list(x0 + displacements, PITCH - chords)
edges3 += [
    (x0 + c, y1),
    (x1, y1),
]
edges3 += make_points_list(x1 -c + displacements, y1 - c + PITCH - chords)
edges3 += [
    (x1, y2),
    (x2, y2),
    (x2, y3),
    (x1, y3),
]
edges3 += make_points_list(x1 - c + displacements, y3 - c + L4 - chords)
edges3 += [
    (x1, y4),
    (x0 + c, y4),
]
poly3 = np.array(edges3)

####
# Make the side helper electrode
####

edges4 = [
    (c, c),
    (c, PITCH-c),
    (PITCH-c, PITCH-c),
] + \
make_points_list(PITCH - displacements, chords) + \
[
    (PITCH - c, c),
    (c, c)
]
poly4 = np.array(edges4) + [0.5*PITCH, PITCH]
poly5 = np.array(edges4) + (0.5*PITCH, -PITCH)

####
# Plot it
####
ax = None
fig = plt.figure()
offsets = np.array([[0, 0], [0, PITCH], [PITCH, 0], [PITCH, PITCH]])
ax = fig.add_subplot(111, sharex=ax)
ax.add_patch(patches.Polygon(poly1))
ax.add_patch(patches.Polygon(poly2,  color='red'))
ax.add_patch(patches.Polygon(poly3,  color='green'))
ax.add_patch(patches.Polygon(poly4,  color='orange'))
ax.add_patch(patches.Polygon(poly5,  color='orange'))
ax.set_xlim([-1, 12])
ax.set_ylim([8, -4])

####
# Write it to kicad footprint file
####
import KicadModTree as kmt
footprint_name = "reservoir_B_%.2fmm" % PITCH
kicad_mod = kmt.Footprint(footprint_name)
kicad_mod.setDescription("Reservoir B for %.2fmm pitch electrodes" % PITCH)

kicad_mod.append(kmt.Text(type="reference", text="", at=[0, -3], layer='F.SilkS'))
kicad_mod.append(kmt.Text(type="value", text="DMFReservoir", at=[1.5, 3], layer='F.Fab'))

def make_pad(number, poly, origin):
    fp_poly = kmt.Polygon(
        nodes=(poly - np.array([PITCH/2, PITCH/2]) - np.array(origin)).tolist(), 
        layer='F.Cu',
        width=0.0001,
    )
    
    kicad_mod.append(kmt.Pad(
        number=number,
        type=kmt.Pad.TYPE_SMT,
        shape=kmt.Pad.SHAPE_CUSTOM,
        at=origin,
        size=[0.5, 0.5],
        layers=kmt.Pad.LAYERS_SMT,
        primitives=[fp_poly],
    ))
    
make_pad(1, poly1, (0, 0))
make_pad(2, poly2, (1.5*PITCH, 0))
make_pad(3, poly3, (3.0*PITCH, 0))
make_pad(4, poly4, (0.5*PITCH, PITCH))
make_pad(5, poly5, (0.5*PITCH, -PITCH))


# output kicad model
file_handler = kmt.KicadFileHandler(kicad_mod)
file_handler.writeFile(footprint_name + ".kicad_mod")


/Users/jeff/venvs/cv4/lib/python3.7/site-packages/ipykernel_launcher.py:101: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
a = np.array([2, 4])
a + [1, 1]

array([3, 5])